In [29]:

from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path

In [9]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cpu"
)  # or device="cpu" if you don't have a GPU

In [10]:
df = pd.read_json("../data/startups_demo.json", lines=True)

In [14]:
vectors = model.encode(
    [row.alt + ". " + row.description for row in df.itertuples()],
    show_progress_bar=True,
)

# Load to vector db

In [16]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qdrant_client = QdrantClient("http://host.docker.internal:6333")

In [24]:
qdrant_client.recreate_collection(
    collection_name="startups", 
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    ) 

True

In [37]:
Path("../data/startup_vectors.npy").exists(), Path("../data/startups_demo.json").exists()

(True, True)

In [25]:
files = open("../data/startups_demo.json", "r")

In [38]:
st_iter = map(json.loads, files)
vectors = np.load("../data/startup_vectors.npy")

In [40]:
qdrant_client.upload_collection(
    collection_name="startups",
    vectors=vectors,
    payload=st_iter,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256,  # How many vectors will be uploaded in a single request?
)